In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf 
import numpy as np
# from layers.py import MaxPoolingWithArgmax2D, MaxUnpooling2D

In [2]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer, Lambda, Conv2D
from tensorflow.keras import initializers, regularizers, constraints, activations

In [3]:
def normalize_tuple(value, n, name):
    """Transforms a single int or iterable of ints into an int tuple.
    # Arguments
        value: The value to validate and convert. Could be an int, or any iterable
          of ints.
        n: The size of the tuple to be returned.
        name: The name of the argument being validated, e.g. `strides` or
          `kernel_size`. This is only used to format error messages.
    # Returns
        A tuple of n integers.
    # Raises
        ValueError: If something else than an int/long or iterable thereof was
        passed.
    """
    if isinstance(value, int):
        return (value,) * n
    else:
        try:
            value_tuple = tuple(value)
        except TypeError:
            raise ValueError('The `{}` argument must be a tuple of {} '
                             'integers. Received: {}'.format(name, n, value))
        if len(value_tuple) != n:
            raise ValueError('The `{}` argument must be a tuple of {} '
                             'integers. Received: {}'.format(name, n, value))
        for single_value in value_tuple:
            try:
                int(single_value)
            except ValueError:
                raise ValueError('The `{}` argument must be a tuple of {} '
                                 'integers. Received: {} including element {} '
                                 'of type {}'.format(name, n, value, single_value,
                                                     type(single_value)))
    return value_tuple


def normalize_padding(value):
    padding = value.lower()
    allowed = {'valid', 'same', 'causal'}
    if K.backend() == 'theano':
        allowed.add('full')
    if padding not in allowed:
        raise ValueError('The `padding` argument must be one of "valid", "same" '
                         '(or "causal" for Conv1D). Received: {}'.format(padding))
    return padding

In [4]:

class MaxPoolingWithArgmax2D(Layer):
    '''MaxPooling for unpooling with indices.
    
    # References
        [SegNet: A Deep Convolutional Encoder-Decoder Architecture for Image Segmentation](http://arxiv.org/abs/1511.00561)
    
    # related code:
        https://github.com/PavlosMelissinos/enet-keras
        https://github.com/ykamikawa/SegNet
    '''
    def __init__(self, pool_size=(2, 2), strides=(2, 2), padding='same', **kwargs):
        super(MaxPoolingWithArgmax2D, self).__init__(**kwargs)
        self.pool_size = normalize_tuple(pool_size, 2, 'pool_size')
        self.strides = normalize_tuple(strides, 2, 'strides')
        self.padding = normalize_padding(padding)

    def call(self, inputs, **kwargs):
        ksize = [1, self.pool_size[0], self.pool_size[1], 1]
        strides = [1, self.strides[0], self.strides[1], 1]
        padding = self.padding.upper()
        output, argmax = tf.nn.max_pool_with_argmax(inputs, ksize, strides, padding)
        argmax = tf.cast(argmax, K.floatx())
        return [output, argmax]
    
    def compute_output_shape(self, input_shape):
        ratio = (1, 2, 2, 1)
        output_shape = [dim // ratio[idx] if dim is not None else None for idx, dim in enumerate(input_shape)]
        output_shape = tuple(output_shape)
        return [output_shape, output_shape]

    def compute_mask(self, inputs, mask=None):
        return 2 * [None]



class MaxUnpooling2D(Layer):
    '''Inversion of MaxPooling with indices.
    
    # References
        [SegNet: A Deep Convolutional Encoder-Decoder Architecture for Image Segmentation](http://arxiv.org/abs/1511.00561)
    
    # related code:
        https://github.com/PavlosMelissinos/enet-keras
        https://github.com/ykamikawa/SegNet
    '''
    def __init__(self, size=(2, 2), **kwargs):
        super(MaxUnpooling2D, self).__init__(**kwargs)
        self.size = normalize_tuple(size, 2, 'size')

    def call(self, inputs, output_shape=None):
        updates, mask = inputs[0], inputs[1]
        
        mask = tf.cast(mask, 'int32')
        input_shape = tf.shape(updates, out_type='int32')
        #  calculation new shape
        if output_shape is None:
            output_shape = (input_shape[0], input_shape[1] * self.size[0], input_shape[2] * self.size[1], input_shape[3])
        
        # calculation indices for batch, height, width and feature maps
        one_like_mask = K.ones_like(mask, dtype='int32')
        batch_shape = K.concatenate([[input_shape[0]], [1], [1], [1]], axis=0)
        batch_range = K.reshape(tf.range(output_shape[0], dtype='int32'), shape=batch_shape)
        b = one_like_mask * batch_range
        y = mask // (output_shape[2] * output_shape[3])
        x = (mask // output_shape[3]) % output_shape[2]
        feature_range = tf.range(output_shape[3], dtype='int32')
        f = one_like_mask * feature_range
        
        # transpose indices & reshape update values to one dimension
        updates_size = tf.size(updates)
        indices = K.transpose(K.reshape(K.stack([b, y, x, f]), [4, updates_size]))
        values = K.reshape(updates, [updates_size])
        ret = tf.scatter_nd(indices, values, output_shape)
        return ret
    
    def compute_output_shape(self, input_shape):
        mask_shape = input_shape[1]
        output_shape = [mask_shape[0], mask_shape[1] * self.size[0], mask_shape[2] * self.size[1], mask_shape[3]]
        return tuple(output_shape)


In [5]:
def segnet(
        input_shape,
        n_labels,
        kernel=3,
        pool_size=(2, 2),
        output_mode="softmax"):
    # encoder
    inputs = tf.keras.Input(shape=input_shape)

    conv_1 = tf.keras.layers.Conv2D(64, (kernel, kernel), padding="same")(inputs)
    conv_1 = tf.keras.layers.BatchNormalization()(conv_1)
    conv_1 = tf.keras.layers.Activation("relu")(conv_1)
    conv_2 = tf.keras.layers.Conv2D(64, (kernel, kernel), padding="same")(conv_1)
    conv_2 = tf.keras.layers.BatchNormalization()(conv_2)
    conv_2 = tf.keras.layers.Activation("relu")(conv_2)

    pool_1, mask_1 = MaxPoolingWithArgmax2D(pool_size)(conv_2)

    conv_3 = tf.keras.layers.Conv2D(128, (kernel, kernel), padding="same")(pool_1)
    conv_3 = tf.keras.layers.BatchNormalization()(conv_3)
    conv_3 = tf.keras.layers.Activation("relu")(conv_3)
    conv_4 = tf.keras.layers.Conv2D(128, (kernel, kernel), padding="same")(conv_3)
    conv_4 = tf.keras.layers.BatchNormalization()(conv_4)
    conv_4 = tf.keras.layers.Activation("relu")(conv_4)

    pool_2, mask_2 = MaxPoolingWithArgmax2D(pool_size)(conv_4)

    conv_5 = tf.keras.layers.Conv2D(256, (kernel, kernel), padding="same")(pool_2)
    conv_5 = tf.keras.layers.BatchNormalization()(conv_5)
    conv_5 = tf.keras.layers.Activation("relu")(conv_5)
    conv_6 = tf.keras.layers.Conv2D(256, (kernel, kernel), padding="same")(conv_5)
    conv_6 = tf.keras.layers.BatchNormalization()(conv_6)
    conv_6 = tf.keras.layers.Activation("relu")(conv_6)
    conv_7 = tf.keras.layers.Conv2D(256, (kernel, kernel), padding="same")(conv_6)
    conv_7 = tf.keras.layers.BatchNormalization()(conv_7)
    conv_7 = tf.keras.layers.Activation("relu")(conv_7)

    pool_3, mask_3 = MaxPoolingWithArgmax2D(pool_size)(conv_7)

    conv_8 = tf.keras.layers.Conv2D(512, (kernel, kernel), padding="same")(pool_3)
    conv_8 = tf.keras.layers.BatchNormalization()(conv_8)
    conv_8 = tf.keras.layers.Activation("relu")(conv_8)
    conv_9 = tf.keras.layers.Conv2D(512, (kernel, kernel), padding="same")(conv_8)
    conv_9 = tf.keras.layers.BatchNormalization()(conv_9)
    conv_9 = tf.keras.layers.Activation("relu")(conv_9)
    conv_10 = tf.keras.layers.Conv2D(512, (kernel, kernel), padding="same")(conv_9)
    conv_10 = tf.keras.layers.BatchNormalization()(conv_10)
    conv_10 = tf.keras.layers.Activation("relu")(conv_10)

    pool_4, mask_4 = MaxPoolingWithArgmax2D(pool_size)(conv_10)

    conv_11 = tf.keras.layers.Conv2D(512, (kernel, kernel), padding="same")(pool_4)
    conv_11 = tf.keras.layers.BatchNormalization()(conv_11)
    conv_11 = tf.keras.layers.Activation("relu")(conv_11)
    conv_12 = tf.keras.layers.Conv2D(512, (kernel, kernel), padding="same")(conv_11)
    conv_12 = tf.keras.layers.BatchNormalization()(conv_12)
    conv_12 = tf.keras.layers.Activation("relu")(conv_12)
    conv_13 = tf.keras.layers.Conv2D(512, (kernel, kernel), padding="same")(conv_12)
    conv_13 = tf.keras.layers.BatchNormalization()(conv_13)
    conv_13 = tf.keras.layers.Activation("relu")(conv_13)

    pool_5, mask_5 = MaxPoolingWithArgmax2D(pool_size)(conv_13)
    print("Build enceder done.")

    # decoder

    unpool_1 = MaxUnpooling2D(pool_size)([pool_5, mask_5])

    conv_14 = tf.keras.layers.Conv2D(512, (kernel, kernel), padding="same")(unpool_1)
    conv_14 = tf.keras.layers.BatchNormalization()(conv_14)
    conv_14 = tf.keras.layers.Activation("relu")(conv_14)
    conv_15 = tf.keras.layers.Conv2D(512, (kernel, kernel), padding="same")(conv_14)
    conv_15 = tf.keras.layers.BatchNormalization()(conv_15)
    conv_15 = tf.keras.layers.Activation("relu")(conv_15)
    conv_16 = tf.keras.layers.Conv2D(512, (kernel, kernel), padding="same")(conv_15)
    conv_16 = tf.keras.layers.BatchNormalization()(conv_16)
    conv_16 = tf.keras.layers.Activation("relu")(conv_16)

    unpool_2 = MaxUnpooling2D(pool_size)([conv_16, mask_4])

    conv_17 = tf.keras.layers.Conv2D(512, (kernel, kernel), padding="same")(unpool_2)
    conv_17 = tf.keras.layers.BatchNormalization()(conv_17)
    conv_17 = tf.keras.layers.Activation("relu")(conv_17)
    conv_18 = tf.keras.layers.Conv2D(512, (kernel, kernel), padding="same")(conv_17)
    conv_18 = tf.keras.layers.BatchNormalization()(conv_18)
    conv_18 = tf.keras.layers.Activation("relu")(conv_18)
    conv_19 = tf.keras.layers.Conv2D(256, (kernel, kernel), padding="same")(conv_18)
    conv_19 = tf.keras.layers.BatchNormalization()(conv_19)
    conv_19 =tf.keras.layers.Activation("relu")(conv_19)

    unpool_3 = MaxUnpooling2D(pool_size)([conv_19, mask_3])

    conv_20 = tf.keras.layers.Conv2D(256, (kernel, kernel), padding="same")(unpool_3)
    conv_20 = tf.keras.layers.BatchNormalization()(conv_20)
    conv_20 = tf.keras.layers.Activation("relu")(conv_20)
    conv_21 = tf.keras.layers.Conv2D(256, (kernel, kernel), padding="same")(conv_20)
    conv_21 = tf.keras.layers.BatchNormalization()(conv_21)
    conv_21 = tf.keras.layers.Activation("relu")(conv_21)
    conv_22 = tf.keras.layers.Conv2D(128, (kernel, kernel), padding="same")(conv_21)
    conv_22 = tf.keras.layers.BatchNormalization()(conv_22)
    conv_22 = tf.keras.layers.Activation("relu")(conv_22)

    unpool_4 = MaxUnpooling2D(pool_size)([conv_22, mask_2])

    conv_23 = tf.keras.layers.Conv2D(128, (kernel, kernel), padding="same")(unpool_4)
    conv_23 = tf.keras.layers.BatchNormalization()(conv_23)
    conv_23 = tf.keras.layers.Activation("relu")(conv_23)
    conv_24 = tf.keras.layers.Conv2D(64, (kernel, kernel), padding="same")(conv_23)
    conv_24 = tf.keras.layers.BatchNormalization()(conv_24)
    conv_24 = tf.keras.layers.Activation("relu")(conv_24)

    unpool_5 = MaxUnpooling2D(pool_size)([conv_24, mask_1])

    conv_25 = tf.keras.layers.Conv2D(64, (kernel, kernel), padding="same")(unpool_5)
    conv_25 = tf.keras.layers.BatchNormalization()(conv_25)
    conv_25 = tf.keras.layers.Activation("relu")(conv_25)

    conv_26 = tf.keras.layers.Conv2D(n_labels, (1, 1), padding="valid")(conv_25)
    conv_26 = tf.keras.layers.BatchNormalization()(conv_26)
    conv_26 = Reshape(
            (input_shape[0]*input_shape[1], n_labels),
            input_shape=(input_shape[0], input_shape[1], n_labels))(conv_26)

    outputs = Activation(output_mode)(conv_26)
    print("Build decoder done.")

    model = Model(inputs=inputs, outputs=outputs, name="SegNet")

    return model

In [15]:
size = 2

x = x_in = tf.keras.Input(shape=(32, 32, 10))
x, m = MaxPoolingWithArgmax2D(pool_size=size)(x)
x_p = x
x = MaxUnpooling2D(size=size)([x,m])
model = tf.keras.Model(x_in, [x,m,x_p])

print(model.input_shape)
print(model.output_shape)

(None, 32, 32, 10)
[(None, None, None, None), (None, 16, 16, 10), (None, 16, 16, 10)]


In [10]:
model1 = segnet((200,200,10),3,5)
print(model1.summary())
#model2 = segnet((480,720,1),3,7)
#model2.summary()

Build enceder done.


ValueError: The channel dimension of the inputs should be defined. Found `None`.